In [1]:
import warnings
warnings.filterwarnings('ignore')

import glob
import time
import math
import numpy as np

import mxnet as mx
from mxnet import gluon, autograd
from mxnet.gluon.utils import download

import gluonnlp as nlp

In [2]:
context = mx.cpu()
batch_size = 1
bptt = 35

In [3]:
dataset_name = 'wikitext-2'
train_dataset, val_dataset, test_dataset = [
    nlp.data.WikiText2(
        segment=segment, bos=None, eos='<eos>', skip_empty=False)
    for segment in ['train', 'val', 'test']
]

vocab = nlp.Vocab(
    nlp.data.Counter(train_dataset), padding_token=None, bos_token=None)

In [4]:
train_dataset = nlp.data.TSVDataset('data/train.csv', num_discard_samples=1)
test_dataset = nlp.data.TSVDataset('data/test.csv', num_discard_samples=1)

In [5]:
awd_model_name = 'awd_lstm_lm_1150'
model, vocab = nlp.model.get_model(
    awd_model_name,
    vocab=vocab,
    dataset_name=dataset_name,
    pretrained=True,
    ctx=context)
loss = gluon.loss.SoftmaxCrossEntropyLoss()
print(model)
print(vocab)

AWDRNN(
  (embedding): HybridSequential(
    (0): Embedding(33278 -> 400, float32)
    (1): Dropout(p = 0.65, axes=(0,))
  )
  (encoder): Sequential(
    (0): LSTM(400 -> 1150, TNC)
    (1): LSTM(1150 -> 1150, TNC)
    (2): LSTM(1150 -> 400, TNC)
  )
  (decoder): HybridSequential(
    (0): Dense(400 -> 33278, linear)
  )
)
Vocab(size=33278, unk="<unk>", reserved="['<eos>']")


In [6]:
X = []; ys = []; yp = []; yn = []
for i, (data, subj, pos, neg) in enumerate(train_dataset):
    hidden = model.begin_state(batch_size=batch_size, func=mx.nd.zeros, ctx=context)
    myarray = mx.nd.expand_dims(mx.nd.array(np.asarray(vocab.to_indices(data.split()))),axis=1)
    output, hidden = model(myarray, hidden)
    X.append(hidden[-1][-1])
    ys.append(int(int(subj) < 2))
    yp.append(int(int(pos) < 2))
    yn.append(int(int(neg) < 2))

In [7]:
import pickle
with open("data/train_x", "wb") as fp:
    pickle.dump(X, fp)
with open("data/train_ys", "wb") as fp:
    pickle.dump(ys, fp)
with open("data/train_yp", "wb") as fp:
    pickle.dump(yp, fp)
with open("data/train_yn", "wb") as fp:
    pickle.dump(yn, fp)

In [8]:
X = []; ys = []; yp = []; yn = []
for i, (data, subj, pos, neg) in enumerate(test_dataset):
    hidden = model.begin_state(batch_size=batch_size, func=mx.nd.zeros, ctx=context)
    myarray = mx.nd.expand_dims(mx.nd.array(np.asarray(vocab.to_indices(data.split()))),axis=1)
    output, hidden = model(myarray, hidden)
    X.append(hidden[-1][-1])
    ys.append(int(int(subj) < 2))
    yp.append(int(int(pos) < 2))
    yn.append(int(int(neg) < 2))

In [9]:
import pickle
with open("data/test_x", "wb") as fp:
    pickle.dump(X, fp)
with open("data/test_ys", "wb") as fp:
    pickle.dump(ys, fp)
with open("data/test_yp", "wb") as fp:
    pickle.dump(yp, fp)
with open("data/test_yn", "wb") as fp:
    pickle.dump(yn, fp)